In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./서울시 휴게음식점 인허가 정보.csv', encoding='cp949')
gg_df = pd.read_csv('./경기도 휴게음식점현황.csv', encoding='cp949')

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_13268\549052057.py:1: DtypeWarning: Columns (13,36,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./서울시 휴게음식점 인허가 정보.csv', encoding='cp949')
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_13268\549052057.py:2: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  gg_df = pd.read_csv('./경기도 휴게음식점현황.csv', encoding='cp949')


In [3]:
df.keys()

Index(['개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자', '영업상태코드', '영업상태명', '상세영업상태코드',
       '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자', '재개업일자', '전화번호', '소재지면적',
       '소재지우편번호', '지번주소', '도로명주소', '도로명우편번호', '사업장명', '최종수정일자', '데이터갱신구분',
       '데이터갱신일자', '업태구분명', '좌표정보(X)', '좌표정보(Y)', '위생업태명', '남성종사자수', '여성종사자수',
       '영업장주변구분명', '등급구분명', '급수시설구분명', '총인원', '본사종업원수', '공장사무직종업원수',
       '공장판매직종업원수', '공장생산직종업원수', '건물소유구분명', '보증액', '월세액', '다중이용업소여부', '시설총규모',
       '전통업소지정번호', '전통업소주된음식', '홈페이지'],
      dtype='object')

In [4]:
gg_df.keys()

Index(['시군명', '사업장명', '인허가일자', '소재지도로명주소', '소재지지번주소', '소재지우편번호', '위도', '경도',
       '인허가취소일자', '통합영업상태명', '영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자', '재개업일자',
       '소재지시설전화번호', '소재지면적정보', '업태구분명정보', 'X좌표값', 'Y좌표값', '위생업태명', '남성종사자수',
       '여성종사자수', '영업장주변구분명', '등급구분명', '급수시설구분명', '총종업원수', '본사종업원수',
       '공장사무직종업원수', '공장판매직종업원수', '공장생산직종업원수', '건물소유구분명', '보증액', '월세액',
       '다중이용업소여부', '시설총규모정보', '전통업소지정번호', '전통업소주된음식', '홈페이지URL'],
      dtype='object')

In [5]:
gg_sb_df = gg_df[gg_df['사업장명'].str.contains('스타벅스')]
gg_sb_df = gg_sb_df[['소재지지번주소','사업장명','통합영업상태명', '인허가일자', '폐업일자', 'X좌표값', 'Y좌표값','위도','경도']]
gg_sb_df.reset_index(drop=True, inplace=True)
gg_sb_df = gg_sb_df.rename(columns={'통합영업상태명' : '영업상태', '인허가일자' : '개업일자','소재지지번주소':'지번주소'})

In [6]:
seoul_sb_df = df[df['사업장명'].str.contains('스타벅스')]
seoul_sb_df = seoul_sb_df[['지번주소','사업장명','상세영업상태명', '인허가일자', '폐업일자', '좌표정보(X)', '좌표정보(Y)']]
seoul_sb_df.reset_index(drop=True, inplace=True)
seoul_sb_df = seoul_sb_df.rename(columns={'좌표정보(X)':'X좌표값','좌표정보(Y)':'Y좌표값', '상세영업상태명' : '영업상태', '인허가일자' : '개업일자'})

In [7]:
seoul_sb_df['개업일자'] = seoul_sb_df['개업일자'].str.replace('-','')
seoul_sb_df['개업일자'] = seoul_sb_df['개업일자'].str[:6]
seoul_sb_df['폐업일자'] = seoul_sb_df['폐업일자'].str[:6]
seoul_sb_df['폐업일자'] = seoul_sb_df['폐업일자'].fillna('202301')
seoul_sb_df['폐업일자'] = seoul_sb_df['폐업일자'].str.replace('-','')

gg_sb_df['개업일자'] = gg_sb_df['개업일자'].str.replace('-','')
gg_sb_df['개업일자'] = gg_sb_df['개업일자'].str[:6]
gg_sb_df['폐업일자'] = gg_sb_df['폐업일자'].str[:6]
gg_sb_df['폐업일자'] = gg_sb_df['폐업일자'].fillna('202301')
gg_sb_df['폐업일자'] = gg_sb_df['폐업일자'].str.replace('-','')
gg_sb_df['영업상태'] = gg_sb_df['영업상태'].str.replace('영업/정상','영업')

In [8]:
splited = seoul_sb_df['지번주소'].str.split(' ', expand=True)
splited[3] = splited[3].replace('산','1-3')
splited[3] = splited[3].str.replace('번지','')
new_splited = splited.loc[:, 0:3]
seoul_sb_df = pd.concat([new_splited,seoul_sb_df], axis=1)

In [9]:
seoul_sb_df = seoul_sb_df.drop(['지번주소'], axis=1)

In [10]:
seoul_sb_df = seoul_sb_df.rename(columns={0 : '서울/경기', 1 : '시/군/구', 2:'동', 3:'번지'})

In [11]:
seoul_sb_df.head()

,서울/경기,시/군/구,동,번지,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값
0,서울특별시,마포구,동교동,173-14,스타벅스 홍대동교점,영업,202303,202301,193414.6768,450675.0036
1,서울특별시,서초구,서초동,1623-1,스타벅스 서울교대점,영업,201007,202301,201279.4432,442806.7092
2,서울특별시,서초구,반포동,20-45,스타벅스 반포자이점,영업,202201,202301,200800.4170,444989.8743
3,서울특별시,강남구,대치동,922-1,스타벅스 대치한티점,영업,202101,202301,204537.4607,444036.1667
4,서울특별시,강남구,대치동,599,스타벅스 대치재경빌딩B1,영업,202012,202301,205464.7204,443592.9735


In [12]:
gg_splited = gg_sb_df['지번주소'].str.split(' ', expand=True)
gg_splited.sample(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
378,경기도,이천시,사음동,572-1,,None,None,None,None,None,None,None,None,None,None,None
229,경기도,수원시,권선구,금곡동,1077,수원모아미래도센트럴타운2단지,B232~234,,None,None,None,None,None,None,None,None
118,경기도,남양주시,별내동,1015,"외1필지오션일레븐4타워제1동101,102,201,202,203호",,None,None,None,None,None,None,None,None,None,None
400,경기도,평택시,안중읍,현화리,649-4,,None,None,None,None,None,None,None,None,None,None
352,경기도,용인시,수지구,성복동,76-2,"1,2층",,None,None,None,None,None,None,None,None,None


In [13]:
gg_splited.loc[gg_splited[2].str[-1] == '구', 1] = gg_splited.loc[gg_splited[2].str[-1] == '구', 1] + gg_splited.loc[gg_splited[2].str[-1] == '구', 2]
gg_splited.loc[gg_splited[2].str[-1] == '구', 2] = gg_splited.loc[gg_splited[2].str[-1] == '구', 3]

In [14]:
gg_splited.loc[gg_splited[1].str[-1] == '구', 3] = ''

In [15]:
gg_splited_n = gg_splited.copy()

In [16]:
gg_splited_n

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,경기도,가평군,가평읍,달전리,113-4,"1층~4층,옥상광장",,None,None,None,None,None,None,None,None,None
1,경기도,고양시일산동구,장항동,,768번지,외3필지.(라페스타B동,"101,120호)",None,None,None,None,None,None,None,None,None
2,경기도,고양시일산동구,풍동,,699,,None,None,None,None,None,None,None,None,None,None
3,경기도,고양시덕양구,덕은동,,0,위프라임트윈타워,,None,None,None,None,None,None,None,None,None
4,경기도,고양시덕양구,원흥동,,332-1,,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,경기도,화성시,반송동,42-2,"(주)반석프라자104호,105호,106호,107호",,None,None,None,None,None,None,None,None,None,None
457,경기도,화성시,반송동,106-1,"(동탄파라곤Ⅱ110,111,112,113호)",,None,None,None,None,None,None,None,None,None,None
458,경기도,화성시,반월동,633-10,,None,None,None,None,None,None,None,None,None,None,None
459,경기도,화성시,기안동,333-5,스타벅스(기안점),"1,2층",전체호,None,None,None,None,None,None,None,None,None


In [17]:
mask = gg_splited_n[3].str[-1].str.isdigit() | (gg_splited_n[3].str[-2:] == '번지')
gg_splited_n[4] = np.where(mask, gg_splited_n[3], gg_splited_n[4])
gg_splited_n.loc[gg_splited_n[3] == '', 4] = gg_splited[4]
gg_splited_n[3] = np.where(mask, '', gg_splited_n[3])

In [18]:
gg_splited_n[4] = gg_splited_n[4].str.replace('번지','')

In [19]:
gg_splited_n[4].unique()

array(['113-4', '768', '699', '0', '332-1', '999', '529', '814', '890-5',
       '1296-3', '370', '373-9', '1050-185', '469', '142-84', '1640',
       '879-2', '1836-3', '81', '18-2', '2602', '973', '689-19', '2700',
       '2222-3', '770-1', '17', '291', '1755', '630', '868', '855', '609',
       '106-3', '568', '628-2', '1332', '620-1', '71-1', '1084-1', '2703',
       '1809', '762', '1237', '751', '1531', '970-5', '1-19', '40-10',
       '79', '425', '500', '158-86', '1077', '1335', '512-3', '513',
       '482', '35', '53-3', '245-9', '12-12', '701-5', '168', '20-14',
       '261-3', '399-6', '526-3', '854-2', '344-6', '874-5', '567-1',
       '383-101', '596', '916', '111', '1145-6', '1132-3', '1099-1',
       '1142-5', '264-4', '774-4', '646-1', '917', '2083-2', '1298-1',
       '2031', '654', '57-1', '326-3', '502-20', '6886-6', '246-6',
       '6880-9', '1608', '620-11', '62-25', '146-26', '59-3', '230-6',
       '97-3', '6147-2', '4001-29', '6216-4', '6058-1', '806-1',
       '

In [20]:
gg_splited_n

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,경기도,가평군,가평읍,달전리,113-4,"1층~4층,옥상광장",,None,None,None,None,None,None,None,None,None
1,경기도,고양시일산동구,장항동,,768,외3필지.(라페스타B동,"101,120호)",None,None,None,None,None,None,None,None,None
2,경기도,고양시일산동구,풍동,,699,,None,None,None,None,None,None,None,None,None,None
3,경기도,고양시덕양구,덕은동,,0,위프라임트윈타워,,None,None,None,None,None,None,None,None,None
4,경기도,고양시덕양구,원흥동,,332-1,,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,경기도,화성시,반송동,,42-2,,None,None,None,None,None,None,None,None,None,None
457,경기도,화성시,반송동,,106-1,,None,None,None,None,None,None,None,None,None,None
458,경기도,화성시,반월동,,633-10,None,None,None,None,None,None,None,None,None,None,None
459,경기도,화성시,기안동,,333-5,"1,2층",전체호,None,None,None,None,None,None,None,None,None


In [21]:
gg_new_splited = gg_splited_n.loc[:, 0:4]
gg_sb_df = pd.concat([gg_new_splited,gg_sb_df], axis=1)
gg_sb_df = gg_sb_df.drop(['지번주소'], axis=1)

In [22]:
gg_sb_df[1] = gg_sb_df[1].str.replace('고양시','고양')
gg_sb_df[1] = gg_sb_df[1].str.replace('성남시','성남')
gg_sb_df[1] = gg_sb_df[1].str.replace('수원시','수원')
gg_sb_df[1] = gg_sb_df[1].str.replace('안산시','안산')
gg_sb_df[1] = gg_sb_df[1].str.replace('안양시','안양')
gg_sb_df[1] = gg_sb_df[1].str.replace('용인시','용인')

In [23]:
gg_sb_df = gg_sb_df.rename(columns={0 : '서울/경기', 1 : '시/군/구', 2:'동', 3:'리', 4:'번지'})

In [24]:
gg_sb_df['동'] = gg_sb_df['동'].str.replace('매산로1가','매산동')

In [25]:
gg_sb_df['번지'].unique()

array(['113-4', '768', '699', '0', '332-1', '999', '529', '814', '890-5',
       '1296-3', '370', '373-9', '1050-185', '469', '142-84', '1640',
       '879-2', '1836-3', '81', '18-2', '2602', '973', '689-19', '2700',
       '2222-3', '770-1', '17', '291', '1755', '630', '868', '855', '609',
       '106-3', '568', '628-2', '1332', '620-1', '71-1', '1084-1', '2703',
       '1809', '762', '1237', '751', '1531', '970-5', '1-19', '40-10',
       '79', '425', '500', '158-86', '1077', '1335', '512-3', '513',
       '482', '35', '53-3', '245-9', '12-12', '701-5', '168', '20-14',
       '261-3', '399-6', '526-3', '854-2', '344-6', '874-5', '567-1',
       '383-101', '596', '916', '111', '1145-6', '1132-3', '1099-1',
       '1142-5', '264-4', '774-4', '646-1', '917', '2083-2', '1298-1',
       '2031', '654', '57-1', '326-3', '502-20', '6886-6', '246-6',
       '6880-9', '1608', '620-11', '62-25', '146-26', '59-3', '230-6',
       '97-3', '6147-2', '4001-29', '6216-4', '6058-1', '806-1',
       '

In [26]:
seoul_sb_df.head()

,서울/경기,시/군/구,동,번지,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값
0,서울특별시,마포구,동교동,173-14,스타벅스 홍대동교점,영업,202303,202301,193414.6768,450675.0036
1,서울특별시,서초구,서초동,1623-1,스타벅스 서울교대점,영업,201007,202301,201279.4432,442806.7092
2,서울특별시,서초구,반포동,20-45,스타벅스 반포자이점,영업,202201,202301,200800.4170,444989.8743
3,서울특별시,강남구,대치동,922-1,스타벅스 대치한티점,영업,202101,202301,204537.4607,444036.1667
4,서울특별시,강남구,대치동,599,스타벅스 대치재경빌딩B1,영업,202012,202301,205464.7204,443592.9735


In [27]:
gg_sb_df.head()

,서울/경기,시/군/구,동,리,번지,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값,위도,경도
0,경기도,가평군,가평읍,달전리,113-4,스타벅스남이섬점,영업,202107,202301,246197.235727,478362.818093,37.806577,127.525344
1,경기도,고양일산동구,장항동,,768,(주)스타벅스커피코리아일산라페스타점,폐업,200308,201306,179538.773119,462032.199846,37.660351,126.768854
2,경기도,고양일산동구,풍동,,699,스타벅스 고양대로DT점,영업,202206,202301,181927.758553,463930.090165,37.677671,126.796117
3,경기도,고양덕양구,덕은동,,0,스타벅스 고양덕은점,영업,202303,202301,NaN,NaN,37.579442,126.870657
4,경기도,고양덕양구,원흥동,,332-1,스타벅스 고양원흥DT점,영업,202108,202301,188391.022888,459392.615118,37.636805,126.869312


In [28]:
# seoul_sb_df.to_csv('./seoul_sb_처리중_2.csv', index=False)
# gg_sb_df.to_csv('./gg_sb_처리중_2.csv', index=False)

## 1) X좌표, Y좌표 값을 처리해야됨

In [29]:
gg_1 = pd.read_csv('./gg_sb_처리중_2.csv')
seoul_1 = pd.read_csv('./seoul_sb_처리중_2.csv')

In [30]:
import pandas as pd

In [31]:
gg_1.loc[gg_1['X좌표값'].isnull()]

,서울/경기,시/군/구,동,리,번지,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값,위도,경도
3,경기도,고양덕양구,덕은동,NaN,0,스타벅스 고양덕은점,영업,202303,202301,NaN,NaN,37.579442,126.870657
23,경기도,고양덕양구,지축동,NaN,0,스타벅스 지축역점,영업,202212,202301,NaN,NaN,37.649763,126.914286
68,경기도,광주시,신현동,NaN,701-5,스타벅스 경기광주신현DT점,영업,201812,202301,NaN,NaN,37.359865,127.153177
77,경기도,구리시,토평동,NaN,567-1,스타벅스 구리토평DT,영업,202112,202301,NaN,NaN,37.584568,127.147300
86,경기도,김포시,걸포동,NaN,264-4,스타벅스 김포걸포점,영업,202203,202301,NaN,NaN,37.633144,126.701030
102,경기도,남양주시,화도읍,묵현리,62-25,스타벅스 남양주묵현DT,영업,202111,202301,NaN,NaN,37.663230,127.302738
111,경기도,남양주시,화도읍,금남리,806-1,스타벅스 더북한강R,영업,202201,202301,NaN,NaN,37.612453,127.342008
112,경기도,남양주시,다산동,NaN,3198-68,스타벅스 도농역점,영업,202004,202301,NaN,NaN,37.611055,127.164624
206,경기도,수원영통구,하동,NaN,871-1,스타벅스 광교갤러리아 9F점,영업,202002,202301,NaN,NaN,37.285334,127.057268
244,경기도,수원영통구,신동,NaN,507-1,스타벅스커피수원트레이더스점,영업,201408,202301,NaN,NaN,37.246109,127.048646


In [32]:
seoul_1.loc[seoul_1['X좌표값'].isnull()]

,서울/경기,시/군/구,동,번지,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값
90,서울특별시,마포구,신공덕동,69-2,스타벅스마포이마트,영업,201201,202301,NaN,NaN
131,서울특별시,송파구,가락동,479,스타벅스 송파헬리오시티점,영업,202004,202301,NaN,NaN
136,서울특별시,용산구,한강로2가,342,스타벅스신용산역점,영업,201708,202301,NaN,NaN
170,서울특별시,중구,남대문로5가,253,스타벅스 숭례문에스지타워,영업,202008,202301,NaN,NaN
171,서울특별시,중구,수표동,47-1,스타벅스 청계크리스탈스퀘어점,영업,202203,202301,NaN,NaN
178,서울특별시,중구,서소문동,75,스타벅스 서소문로점,영업,201410,202301,NaN,NaN
192,서울특별시,용산구,효창동,13-2,스타벅스효창공원앞역점,영업,201911,202301,NaN,NaN
204,서울특별시,용산구,한강로3가,63-70,스타벅스용산해링턴스퀘어점,영업,202012,202301,NaN,NaN
213,서울특별시,중구,을지로2가,88-5,스타벅스 을지로2가점,영업,201605,202301,NaN,NaN
215,서울특별시,서초구,신원동,205-1,스타벅스 청계산입구역점,영업,201603,202301,NaN,NaN


In [33]:
from pyproj import Proj, transform

mask = (gg_1['X좌표값'].isnull()) & (gg_1['Y좌표값'].isnull())
filtered_df = gg_1[mask]

proj_UTMK = Proj(init='epsg:5178')
proj_WGS84 = Proj(init='epsg:4326')

for index, row in filtered_df.iterrows():
    lon, lat = row['경도'], row['위도']
    x, y = transform(proj_WGS84, proj_UTMK, lon, lat)
    gg_1.loc[index, 'X좌표값'] = x
    gg_1.loc[index, 'Y좌표값'] = y

c:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_13268\476233495.py:11: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotcha

In [34]:
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="aa")

def get_geocode(address):
    location = geolocator.geocode(address)
    if location is None:
        return None, None
    else:
        return location.latitude, location.longitude
    
for i, row in seoul_1.loc[seoul_1['X좌표값'].isna() | seoul_1['Y좌표값'].isna()].iterrows():
    address = row['시/군/구'] + ' ' + row['동'] + ' ' + row['번지']
    lat, lon = get_geocode(address)
    seoul_1.at[i, 'X좌표값'] = lon
    seoul_1.at[i, 'Y좌표값'] = lat
    time.sleep(1) 

In [35]:
seoul_1

,서울/경기,시/군/구,동,번지,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값
0,서울특별시,마포구,동교동,173-14,스타벅스 홍대동교점,영업,202303,202301,193414.6768,450675.0036
1,서울특별시,서초구,서초동,1623-1,스타벅스 서울교대점,영업,201007,202301,201279.4432,442806.7092
2,서울특별시,서초구,반포동,20-45,스타벅스 반포자이점,영업,202201,202301,200800.4170,444989.8743
3,서울특별시,강남구,대치동,922-1,스타벅스 대치한티점,영업,202101,202301,204537.4607,444036.1667
4,서울특별시,강남구,대치동,599,스타벅스 대치재경빌딩B1,영업,202012,202301,205464.7204,443592.9735
...,...,...,...,...,...,...,...,...,...,...
738,서울특별시,강동구,천호동,455-8,스타벅스현대천호점,폐업,200406,201408,210931.6435,448522.0798
739,서울특별시,강동구,성내동,468,스타벅스커피 올림픽공원 북문점,폐업,201405,202107,211064.0970,447034.5346
740,서울특별시,강동구,천호동,455-8,스타벅스 현대천호B1점,폐업,201508,201810,210931.6435,448522.0798
741,서울특별시,강동구,길동,459-4,스타벅스 길동사거리점,폐업,201008,202009,212167.3054,448056.9768


In [36]:
import pyproj

in_proj = Proj(init='epsg:5186')
out_proj = Proj(init='epsg:4326')

seoul_1['위도'] = 0
seoul_1['경도'] = 0

for i in range(len(seoul_1)):
    x = seoul_1.at[i, 'X좌표값']
    y = seoul_1.at[i, 'Y좌표값']
    lon, lat = transform(in_proj, out_proj, x, y)
    seoul_1.at[i, '위도'] = lat
    seoul_1.at[i, '경도'] = lon

c:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_13268\790702232.py:12: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotcha

In [37]:
seoul_1

,서울/경기,시/군/구,동,번지,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값,위도,경도
0,서울특별시,마포구,동교동,173-14,스타벅스 홍대동교점,영업,202303,202301,193414.6768,450675.0036,36.654511,126.926349
1,서울특별시,서초구,서초동,1623-1,스타벅스 서울교대점,영업,201007,202301,201279.4432,442806.7092,36.583628,127.014296
2,서울특별시,서초구,반포동,20-45,스타벅스 반포자이점,영업,202201,202301,200800.4170,444989.8743,36.603302,127.008946
3,서울특별시,강남구,대치동,922-1,스타벅스 대치한티점,영업,202101,202301,204537.4607,444036.1667,36.594697,127.050708
4,서울특별시,강남구,대치동,599,스타벅스 대치재경빌딩B1,영업,202012,202301,205464.7204,443592.9735,36.590699,127.061068
...,...,...,...,...,...,...,...,...,...,...,...,...
738,서울특별시,강동구,천호동,455-8,스타벅스현대천호점,폐업,200406,201408,210931.6435,448522.0798,36.635070,127.122230
739,서울특별시,강동구,성내동,468,스타벅스커피 올림픽공원 북문점,폐업,201405,202107,211064.0970,447034.5346,36.621664,127.123689
740,서울특별시,강동구,천호동,455-8,스타벅스 현대천호B1점,폐업,201508,201810,210931.6435,448522.0798,36.635070,127.122230
741,서울특별시,강동구,길동,459-4,스타벅스 길동사거리점,폐업,201008,202009,212167.3054,448056.9768,36.630864,127.136039


In [38]:
seoul_1.isnull().sum()

서울/경기    0
시/군/구    0
동        0
번지       0
사업장명     0
영업상태     0
개업일자     0
폐업일자     0
X좌표값     0
Y좌표값     0
위도       0
경도       0
dtype: int64

In [39]:
gg_1

,서울/경기,시/군/구,동,리,번지,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값,위도,경도
0,경기도,가평군,가평읍,달전리,113-4,스타벅스남이섬점,영업,202107,202301,246197.235727,4.783628e+05,37.806577,127.525344
1,경기도,고양일산동구,장항동,NaN,768,(주)스타벅스커피코리아일산라페스타점,폐업,200308,201306,179538.773119,4.620322e+05,37.660351,126.768854
2,경기도,고양일산동구,풍동,NaN,699,스타벅스 고양대로DT점,영업,202206,202301,181927.758553,4.639301e+05,37.677671,126.796117
3,경기도,고양덕양구,덕은동,NaN,0,스타벅스 고양덕은점,영업,202303,202301,944620.173415,1.953220e+06,37.579442,126.870657
4,경기도,고양덕양구,원흥동,NaN,332-1,스타벅스 고양원흥DT점,영업,202108,202301,188391.022888,4.593926e+05,37.636805,126.869312
...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,경기도,화성시,반송동,NaN,42-2,스타벅스동탄센트럴파크,영업,201501,202301,205603.594214,4.118283e+05,37.208334,127.063894
457,경기도,화성시,반송동,NaN,106-1,스타벅스커피동탄점,영업,201305,202301,206425.845143,4.110853e+05,37.201617,127.073229
458,경기도,화성시,반월동,NaN,633-10,스타벅스화성반월DT점,영업,202006,202301,205074.291306,4.131479e+05,37.220165,127.057931
459,경기도,화성시,기안동,NaN,333-5,스타벅스화성봉담DT점,영업,202009,202301,197417.320170,4.140483e+05,37.228339,126.971579


In [40]:
gg_1.isnull().sum()

서울/경기      0
시/군/구      0
동          0
리        422
번지         0
사업장명       0
영업상태       0
개업일자       0
폐업일자       0
X좌표값       0
Y좌표값       0
위도         0
경도         0
dtype: int64

In [41]:
seoul_1.to_csv('./seoul_sb_처리중_3.csv', index=False)
gg_1.to_csv('./gg_sb_처리중_3.csv', index=False)